## Sliding windows

In [26]:
import pandas as pd
import numpy as np
import time
import scipy.signal

In [27]:
df = pd.read_csv(r'datasets\one-hot-enc.csv')
df.head()

,PDB_ID,CHAIN_ID,RES_ID,LIP_SCORE,LIP,REL_ASA,PHI,PSI,NH_O_1_relidx,NH_O_1_energy,...,MC_SC,NO_EDGE_LOC,SC_MC,SC_SC,HBOND,IAC,NO_EDGE_TYPE,PIPISTACK,VDW,CHAIN_LEN
0,1cee,A,1,0,0,1.000000,360.0,97.6,0.0,0.0,...,0,1,0,0,0,0,1,0,0,179
1,1cee,A,2,0,0,0.348485,-91.3,147.8,48.0,-0.1,...,0,0,0,2,2,0,0,0,1,179
2,1cee,A,3,0,0,0.387324,-142.6,136.7,-2.0,-0.3,...,0,1,0,0,0,0,1,0,0,179
3,1cee,A,4,0,0,0.005917,-94.3,154.4,48.0,-1.9,...,0,0,0,3,2,0,0,0,3,179
4,1cee,A,5,0,0,0.346341,-112.5,70.5,-2.0,-0.2,...,0,0,1,0,1,0,0,0,1,179


In [28]:
LIP = df.LIP
df.drop(['LIP_SCORE', 'LIP'], inplace = True, axis = 1)
df.head(10)

,PDB_ID,CHAIN_ID,RES_ID,REL_ASA,PHI,PSI,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,...,MC_SC,NO_EDGE_LOC,SC_MC,SC_SC,HBOND,IAC,NO_EDGE_TYPE,PIPISTACK,VDW,CHAIN_LEN
0,1cee,A,1,1.000000,360.0,97.6,0.0,0.0,2.0,-0.3,...,0,1,0,0,0,0,1,0,0,179
1,1cee,A,2,0.348485,-91.3,147.8,48.0,-0.1,50.0,-1.7,...,0,0,0,2,2,0,0,0,1,179
2,1cee,A,3,0.387324,-142.6,136.7,-2.0,-0.3,50.0,-0.2,...,0,1,0,0,0,0,1,0,0,179
3,1cee,A,4,0.005917,-94.3,154.4,48.0,-1.9,50.0,-1.6,...,0,0,0,3,2,0,0,0,3,179
4,1cee,A,5,0.346341,-112.5,70.5,-2.0,-0.2,71.0,-1.6,...,0,0,1,0,1,0,0,0,1,179
5,1cee,A,6,0.000000,-86.3,107.8,48.0,-1.8,50.0,-3.0,...,0,0,0,1,2,0,0,0,1,179
6,1cee,A,7,0.000000,-112.8,133.8,70.0,-1.0,72.0,-2.8,...,0,0,0,0,1,0,0,0,0,179
7,1cee,A,8,0.007042,-86.5,115.8,48.0,-2.9,50.0,-0.6,...,0,0,0,2,1,0,0,0,2,179
8,1cee,A,9,0.035211,-113.0,145.4,70.0,-2.2,72.0,-1.1,...,0,0,0,0,1,0,0,0,0,179
9,1cee,A,10,0.071429,154.5,148.8,-2.0,-0.4,2.0,-0.5,...,0,1,0,0,0,0,1,0,0,179


In [29]:
"""
TODO: implementing sliding windows for first end last res in chains, solve why repeating procedure create more NaN
"""

### Apply window of size four to single chains of protein 1cee
df_windows = df
for chain in set(df.CHAIN_ID[df.PDB_ID == '1cee'].unique()):
    info_sliced = df.iloc[:, 0:3]
    sliced = df_windows.iloc[:, 3:][(df_windows.PDB_ID == '1cee') & (df_windows.CHAIN_ID == chain)]
    sliced = sliced.rolling(window = 2, win_type='gaussian', center = True).mean(std=0.5)
    tot_sliced = pd.merge(info_sliced, sliced, right_index=True, left_index=True)
    df_windows[(df_windows.PDB_ID == '1cee') & (df_windows.CHAIN_ID == chain)] = tot_sliced

df_windows.head(200)


,PDB_ID,CHAIN_ID,RES_ID,REL_ASA,PHI,PSI,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,...,MC_SC,NO_EDGE_LOC,SC_MC,SC_SC,HBOND,IAC,NO_EDGE_TYPE,PIPISTACK,VDW,CHAIN_LEN
0,1cee,A,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1cee,A,2,0.674242,134.35,122.70,24.0,-0.05,26.0,-1.00,...,0.0,0.5,0.0,1.0,1.0,0.0,0.5,0.0,0.5,179.0
2,1cee,A,3,0.367904,-116.95,142.25,23.0,-0.20,50.0,-0.95,...,0.0,0.5,0.0,1.0,1.0,0.0,0.5,0.0,0.5,179.0
3,1cee,A,4,0.196621,-118.45,145.55,23.0,-1.10,50.0,-0.90,...,0.0,0.5,0.0,1.5,1.0,0.0,0.5,0.0,1.5,179.0
4,1cee,A,5,0.176129,-103.40,112.45,23.0,-1.05,60.5,-1.60,...,0.0,0.0,0.5,1.5,1.5,0.0,0.0,0.0,2.0,179.0
5,1cee,A,6,0.173171,-99.40,89.15,23.0,-1.00,60.5,-2.30,...,0.0,0.0,0.5,0.5,1.5,0.0,0.0,0.0,1.0,179.0
6,1cee,A,7,0.000000,-99.55,120.80,59.0,-1.40,61.0,-2.90,...,0.0,0.0,0.0,0.5,1.5,0.0,0.0,0.0,0.5,179.0
7,1cee,A,8,0.003521,-99.65,124.80,59.0,-1.95,61.0,-1.70,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,179.0
8,1cee,A,9,0.021127,-99.75,130.60,59.0,-2.55,61.0,-0.85,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,179.0
9,1cee,A,10,0.053320,20.75,147.10,34.0,-1.30,37.0,-0.80,...,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.0,0.0,179.0
